In [1]:
!pip3 install pytorch-transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 65.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=1712df38901f14ea577843c42b8195f21d35ceb804ab64f96bc2ddc896545e43
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [2]:
import torch
import random
import numpy as np
from copy import deepcopy
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [8]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')
file_path = '/content/gdrive/MyDrive/dacon law/'
train_data = pd.read_csv(file_path+'train.csv')
test_data = pd.read_csv(file_path+'test.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
train_data.drop("ID", axis=1, inplace=True)

In [10]:
from sklearn.model_selection import train_test_split
val_rate=0.2
train_indices, val_indices = train_test_split(range(len(train_data)), test_size=val_rate)

In [11]:
from torch.utils.data import DataLoader, Subset
train_dataset = train_data.iloc[train_indices]
validation_dataset = train_data.iloc[val_indices]

train_dataset.reset_index(drop=False, inplace=True)
validation_dataset.reset_index(drop=False, inplace=True)

In [12]:
from transformers.optimization import get_linear_schedule_with_warmup
device = torch.device("cuda")
from transformers import AutoTokenizer, BertForSequenceClassification, AdamW
import transformers
transformers.logging.set_verbosity_error()
from torch import nn
criterion = nn.BCELoss().to(device)
import torch.nn.functional as F
# Define the dataset class
class CaseDataset(torch.utils.data.Dataset):
    def __init__(self, first_parties, second_parties, case_contents, labels):
        self.first_parties = first_parties
        self.second_parties = second_parties
        self.case_contents = case_contents
        self.labels = labels
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.first_parties)

    def __getitem__(self, idx):
        inputs = self.tokenizer.encode_plus(
            self.first_parties[idx],
            self.case_contents[idx],
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        label = torch.tensor(self.labels[idx])
        return {
            'input_ids': torch.tensor(inputs['input_ids'].squeeze(), device=device),
            'attention_mask': torch.tensor(inputs['attention_mask'].squeeze(), device=device),
            'labels': torch.tensor(label, device=device)
        }

# Define training parameters
batch_size = 8
num_epochs = 20
learning_rate = 2e-5

train_first_parties=train_dataset['first_party']
train_second_parties=train_dataset['second_party']
train_case_contents=train_dataset['facts']
train_labels=train_dataset['first_party_winner']

train_dataset_ = CaseDataset(train_first_parties, train_second_parties, train_case_contents, train_labels)
train_dataloader = DataLoader(train_dataset_, batch_size=batch_size, shuffle=True)

total_steps=len(train_dataloader)*num_epochs

validation_first_parties=validation_dataset['first_party']
validation_second_parties=validation_dataset['second_party']
validation_case_contents=validation_dataset['facts']
validation_labels=validation_dataset['first_party_winner']

validation_dataset_ = CaseDataset(validation_first_parties, validation_second_parties, validation_case_contents, train_labels)
validation_dataloader = DataLoader(validation_dataset_, batch_size=batch_size, shuffle=True)


# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

scheduler=get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [14]:
# Training loop
itr=1
p_itr=500
model.train()
total_loss=0
total_correct=0
total_len=0
valid_losses, lowest_loss= list(), np.inf
early_stop = 100
progress_interval=1
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        logits=outputs.logits
        pred=torch.argmax(F.softmax(logits), dim=1)
        correct=pred.eq(labels)
        total_correct+=correct.sum().item()
        total_len+=len(labels)
        total_loss+=loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        itr+=1

    print('[Epoch {}/{}] -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, num_epochs, total_loss/itr, total_correct/total_len))
    total_loss = 0
    total_len = 0
    total_correct = 0
    itr=0


    # validate the model
    model.eval()
    with torch.no_grad():
        for batch in validation_dataloader:
          input_ids = batch['input_ids']
          attention_mask = batch['attention_mask']
          labels = batch['labels']
          outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
          loss = outputs.loss
          valid_losses.append(loss.item())
          logits=outputs.logits
          pred=torch.argmax(F.softmax(logits), dim=1)

    if valid_losses[-1] < lowest_loss:
        lowest_loss = valid_losses[-1]
        lowest_epoch = epoch
        best_model = deepcopy(model.state_dict())
        torch.save(model.state_dict(), file_path + 'BERT_with_scheduler_adamw,2e5.pth')  # 모델 객체의 state_dict 저장
    else:
        if early_stop > 0 and lowest_epoch + early_stop < epoch:
            print("Early Stopped", epoch, "epochs")
            model.load_state_dict(best_model)
            break

    if (epoch % progress_interval) == 0:
        print(valid_losses[-1], lowest_loss, lowest_epoch, epoch)




<ipython-input-12-4ef5f11bea24>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(inputs['input_ids'].squeeze(), device=device),
<ipython-input-12-4ef5f11bea24>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(inputs['attention_mask'].squeeze(), device=device),
<ipython-input-12-4ef5f11bea24>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(label, device=device)
<ipython-input-14-821c90323845>:22: UserWarning: Implicit dimension choice for softmax has been d

[Epoch 1/20] -> Train Loss: 0.6509, Accuracy: 0.636


<ipython-input-14-821c90323845>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred=torch.argmax(F.softmax(logits), dim=1)


0.6709040403366089 0.6709040403366089 0 0
[Epoch 2/20] -> Train Loss: 0.6493, Accuracy: 0.632
0.8154187798500061 0.6709040403366089 0 1
[Epoch 3/20] -> Train Loss: 0.6531, Accuracy: 0.631
0.6151990294456482 0.6151990294456482 2 2
[Epoch 4/20] -> Train Loss: 0.6547, Accuracy: 0.630
0.6833717823028564 0.6151990294456482 2 3
[Epoch 5/20] -> Train Loss: 0.6511, Accuracy: 0.636
0.6359623670578003 0.6151990294456482 2 4
[Epoch 6/20] -> Train Loss: 0.6506, Accuracy: 0.638
0.7031323909759521 0.6151990294456482 2 5
[Epoch 7/20] -> Train Loss: 0.6534, Accuracy: 0.633
0.6359274983406067 0.6151990294456482 2 6
[Epoch 8/20] -> Train Loss: 0.6472, Accuracy: 0.633
0.6009376645088196 0.6009376645088196 7 7
[Epoch 9/20] -> Train Loss: 0.6512, Accuracy: 0.636
0.6882365942001343 0.6009376645088196 7 8
[Epoch 10/20] -> Train Loss: 0.6510, Accuracy: 0.636
0.5476773977279663 0.5476773977279663 9 9
[Epoch 11/20] -> Train Loss: 0.6505, Accuracy: 0.640
0.6098377704620361 0.5476773977279663 9 10
[Epoch 12/20] -

In [15]:
model.load_state_dict(best_model)

<All keys matched successfully>

In [16]:
# Inference on the test set
test_first_parties=test_data['first_party']
test_second_parties=test_data['second_party']
test_case_contents=test_data['facts']

test_dataset = CaseDataset(test_first_parties, test_second_parties, test_case_contents, [0] * len(test_first_parties))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

model.eval()
predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(outputs.logits, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)
        predictions.extend(predicted_labels.tolist())

<ipython-input-12-4ef5f11bea24>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(inputs['input_ids'].squeeze(), device=device),
<ipython-input-12-4ef5f11bea24>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(inputs['attention_mask'].squeeze(), device=device),
<ipython-input-12-4ef5f11bea24>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(label, device=device)


In [17]:
just_test = pd.read_csv(file_path+'test.csv')
df = pd.DataFrame(predictions)
df.transpose()
just_test['first_party_winner']=df

In [18]:
just_test.to_csv('submission_2.csv', sep=',')